In [1]:
import cv2
import numpy as np

In [2]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [3]:
def preProcessing(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (5,5), 1)
    imgCanny = cv2.Canny(imgBlur, 200, 200)
    kernel = np.ones([5,5])
    imgDil = cv2.dilate(imgCanny, kernel, iterations=2)
    imgErod = cv2.erode(imgDil, kernel, iterations=1)
    return imgErod

In [4]:
def getContours(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    maxArea = 0
    biggest = np.array([])
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 5000:
            cv2.drawContours(imgContour, cnt, -1, (255,0,0), 3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True) #corner points
            if area > maxArea and len(approx)==4:
                biggest = approx
                maxArea = area
    # x,y,w,h = cv2.boundingRect(approx)
    # cv2.rectangle(imgContour,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.drawContours(imgContour, biggest, -1, (255,0,0), 30)
    return biggest

In [5]:
def getWarp(img, biggest):
    pt1 = np.float32(biggest)
    pt2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
    matrix = cv2.getPerspectiveTransform(pt1, pt2)
    img_out = cv2.warpPerspective(img, matrix, (width,height))
    return img_out

In [6]:
def reorder(pts):
    pts = pts.reshape((4,2))
    newPts = np.zeros((4,1,2), dtype=np.int32)
    
    add = pts.sum(axis=1)
    newPts[0] = pts[np.argmin(add)]
    newPts[3] = pts[np.argmax(add)]

    diff = np.diff(pts, axis=1)
    newPts[1] = pts[np.argmin(diff)]
    newPts[2] = pts[np.argmax(diff)]
    return newPts

In [9]:
width, height = 640, 480

cap = cv2.VideoCapture(2) # 0 for webcam ,,, 2 for android_cam
cap.set(3, width)      # width
cap.set(4, height)      # height
cap.set(10, 150)    # brightness

while True:
    success, img = cap.read()
    if not success or (cv2.waitKey(1) & 0xFF == ord('q')):
        break
    else:
        imgContour = img.copy()
        imgThres = preProcessing(img)
        biggest = getContours(imgThres)
        
        if biggest.size != 0:
            imgWarpd = getWarp(img, reorder(biggest))
            imgArray = ([imgContour, imgWarpd])
        else:
            imgArray = ([imgContour, img])

        imgStacked = stackImages(1, imgArray)
        cv2.imshow("boundry", imgStacked)
        # cv2.imshow("video", img)
        # cv2.waitKey(10)
    
cap.release()
cv2.destroyAllWindows()